##### Imports

In [1]:
import pandas as pd
import csv
import logging
from tracker_funcs import read_jem, read_ephys, read_shiny, \
sort_df, create_cond_df, create_region_col, merge_dfs, choice

import seaborn as sns

##### File Paths

In [2]:
csv_path = "C:/Users/ramr/Documents/Github/analysis_projects/csv/"
excel_path = "C:/Users/ramr/Documents/Github/analysis_projects/excel/"

##### Logging Set-up

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

##### Miscellaneous Settings

In [4]:
sns.set(context = "notebook", style = "ticks", font="verdana", font_scale = 1.35)
pd.set_option("display.max_colwidth",150) #Expands the number of characters shown in the columns
pd.set_option('display.max_columns', None)

##### Classes

In [5]:
class Dataframe():
    """Dataframe class to uses common methods for pandas dataframes"""
    
    filter_jem_cols = ["date", "organism_name", "name", "container", "rigOperator",
                       "status", "roi_major", "roi_minor",
                       "extraction.postPatch", "extraction.endPipetteR"] 
    
    filter_ephys_cols=["b'patched_cell_container'", "b'vrest'", "b'sag'", "b'tau'",
                       "b'upstroke_downstroke_ratio_long_square'", "b'latency'", "b'f_i_curve_slope'"]
    
    def __init__(self, location, df=pd.DataFrame()):
        """Initializer
        Args:
            df: pandas dataframe
        Returns: None
        """
        self.df = df
        self.location = location #Initializing with self so you can use in any methods
        
    def read_data(self, fields=None):
        """Reads data from a file location
        Args:
            fields: columns to filter the dataframe
        Returns: 
            self.df: pandas dataframe with filtered columns if applied
        """
        self.df = pd.read_csv(self.location, usecols=fields)
        #return self.df
    
    def create_cond_df(self, col_name, cond):
        """Creates a new pandas dataframe based on a conditional statement by a single column
        Args:
            col_name: column name used to filter condition
            cond: condition to filter column by
        Returns: 
            self.df: new pandas dataframe based on conditional statement
        """
        self.df = self.df[self.df[col_name] == cond]
        logger.info("Created a conditional dataframe")
        return self.df
    
    def filter_df(self, col_list):
        """Filters dataframe based on which columns users wants to display as a table
        Args:
            col_list: columns to filter the dataframe
        Returns: 
            self.df: pandas dataframe with filtered columns if applied
        """
        self.df = self.df[col_list]
        return self.df

In [39]:
class Jem(Dataframe):
    """Subclass of Dataframe Class"""
    
    def read_data(self, fields=None):
        """Reads data from a file location
        Args:
            fields: columns to filter the dataframe
        Returns: 
            self.df: pandas dataframe with filtered columns if applied
        """
        self.df = pd.read_csv(self.location, usecols=fields, index_col=["date"])
        logger.info("Read jem metadata csv as a pandas dataframe")
        return self.df

In [36]:
j = Jem("Z:/Patch-Seq/compiled-jem-data/jem_metadata.csv")
jem = j.read_data(j.filter_jem_cols)
jem

INFO:__main__:Read jem metadata csv as a pandas dataframe


,organism_name,name,extraction.endPipetteR,extraction.postPatch,rigOperator,status,roi_major,roi_minor,container
date,,,,,,,,,
2017-10-02 10:46:00 -0700,Mouse,Oxtr-T2A-Cre;Ai14-351467.03.01.01,NaN,nucleus_present,lisak,SUCCESS (high confidence),VISp,layer 2/3,P9S4_171002_401_A01
2017-10-02 10:54:00 -0700,Mouse,Oxtr-T2A-Cre;Ai14-351467.05.02.02,NaN,nucleus_present,lindsayn,SUCCESS (high confidence),VISp,layer 5,P8S4_171002_352_A01
2017-10-02 10:54:00 -0700,Mouse,Oxtr-T2A-Cre;Ai14-351467.05.02.01,NaN,nucleus_present,lindsayn,SUCCESS (high confidence),VISp,layer 5,P8S4_171002_351_A01
2017-10-02 10:54:00 -0700,Mouse,Oxtr-T2A-Cre;Ai14-351467.05.02.03,NaN,nucleus_absent,lindsayn,SUCCESS (high confidence),VISp,layer 5,P8S4_171002_353_A01
2017-10-02 11:30:00 -0700,Mouse,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-350934.04.01.02,NaN,nucleus_present,rustym,SUCCESS (high confidence),VISp,layer 2/3,P2S4_171002_052_A01
...,...,...,...,...,...,...,...,...,...
NaN,Human,H18.28.015.11.09.01,NaN,NaN,NaN,NaN,NaN,NaN,PGS4_180427_904_A01
NaN,Human,H20.03.303.11.54.01.02.06,NaN,NaN,NaN,NaN,NaN,NaN,PTS4_200207_232_A01
NaN,Mouse,Sst-IRES-Cre;Pvalb-T2A-FlpO;Ai193-hyg-492717.08.06.01.02,NaN,NaN,NaN,NaN,NaN,NaN,PMS4_191007_025_A01


In [40]:
jem = j.date_filter()

TypeError: date_filter() takes 0 positional arguments but 1 was given

In [46]:
r_users = ["kristenh", "lindsayn", "ramr", "katherineb", "jessicat"] 
jem = jem.loc["2020-01-03 10:40:30 -0800":,:]

jem = jem[jem.index.notnull()]
jem.dropna(subset=["rigOperator", "container"], inplace=True)
logger.info("Dropped NaNs from index date, rigOperator and container column")

jem = jem[jem["status"] == "SUCCESS"]
jem = jem[jem.rigOperator.str.contains("|".join(r_users))]
logger.info("Created dataframe with selected users in container column")

INFO:__main__:Dropped NaNs from index date, rigOperator and container column
INFO:__main__:Created dataframe with selected users in container column


In [47]:
jem

,organism_name,name,extraction.endPipetteR,extraction.postPatch,rigOperator,status,roi_major,roi_minor,container
date,,,,,,,,,
2020-01-03 10:40:30 -0800,Mouse,Gad2-IRES-Cre;Ai14-504539.10.01.01,1000.0,nucleus_present,jessicat,SUCCESS,RSPd,layer 2/3,PFS4_200103_701_A01
2020-01-03 13:00:07 -0800,Mouse,Pvalb-IRES-Cre;Ai14-504851.11.01.02,2000.0,nucleus_present,jessicat,SUCCESS,RSPv,layer 2/3,PFS4_200103_703_A01
2020-01-03 13:00:07 -0800,Mouse,Pvalb-IRES-Cre;Ai14-504851.11.01.01,1000.0,nucleus_present,jessicat,SUCCESS,RSPd,layer 5,PFS4_200103_702_A01
2020-01-06 13:12:44 -0800,Mouse,Vip-IRES-Cre;Ai14-504179.11.01.02,3000.0,nucleus_present,jessicat,SUCCESS,SSp,layer 5,PFS4_200106_702_A01
2020-01-06 13:12:44 -0800,Mouse,Vip-IRES-Cre;Ai14-504179.11.01.01,7.0,nucleus_present,jessicat,SUCCESS,RSPd,layer 5,PFS4_200106_701_A01
...,...,...,...,...,...,...,...,...,...
2020-02-11 13:46:18 -0800,Mouse,Slc32a1-IRES-Cre;Sncg-IRES2-FlpO;Ai65-509370.03.01.02,4500.0,nucleus_present,kristenh,SUCCESS,HIP,CA1,P1S4_200211_004_A01
2020-02-11 13:50:40 -0800,Mouse,Slc32a1-IRES-Cre;Sncg-IRES2-FlpO;Ai65-509370.03.02.02,800.0,nucleus_present,lindsayn,SUCCESS,HIP,CA1,P8S4_200211_354_A01
2020-02-11 13:50:40 -0800,Mouse,Slc32a1-IRES-Cre;Sncg-IRES2-FlpO;Ai65-509370.03.02.01,2000.0,nucleus_present,lindsayn,SUCCESS,HIP,CA1,P8S4_200211_353_A01


In [13]:
m = j.create_cond_df("organism_name", "Mouse")
m

INFO:__main__:Created a conditional dataframe


,organism_name,name,date,extraction.endPipetteR,extraction.postPatch,rigOperator,status,roi_major,roi_minor,container
0,Mouse,Oxtr-T2A-Cre;Ai14-351467.03.01.01,2017-10-02 10:46:00 -0700,NaN,nucleus_present,lisak,SUCCESS (high confidence),VISp,layer 2/3,P9S4_171002_401_A01
1,Mouse,Oxtr-T2A-Cre;Ai14-351467.05.02.02,2017-10-02 10:54:00 -0700,NaN,nucleus_present,lindsayn,SUCCESS (high confidence),VISp,layer 5,P8S4_171002_352_A01
2,Mouse,Oxtr-T2A-Cre;Ai14-351467.05.02.01,2017-10-02 10:54:00 -0700,NaN,nucleus_present,lindsayn,SUCCESS (high confidence),VISp,layer 5,P8S4_171002_351_A01
3,Mouse,Oxtr-T2A-Cre;Ai14-351467.05.02.03,2017-10-02 10:54:00 -0700,NaN,nucleus_absent,lindsayn,SUCCESS (high confidence),VISp,layer 5,P8S4_171002_353_A01
4,Mouse,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-350934.04.01.02,2017-10-02 11:30:00 -0700,NaN,nucleus_present,rustym,SUCCESS (high confidence),VISp,layer 2/3,P2S4_171002_052_A01
...,...,...,...,...,...,...,...,...,...,...
12593,Mouse,HM_Lab_Test_m.17.11.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHS4_180726_101_A01
12594,Mouse,Sst-IRES-Cre;Pvalb-T2A-FlpO;Ai193-hyg-490656.08.06.01.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PMS4_191001_027_A01
12600,Mouse,GT_Lab_Test.02.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGS4_180119_006_A01
12604,Mouse,Sst-IRES-Cre;Pvalb-T2A-FlpO;Ai193-hyg-492717.08.06.01.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PMS4_191007_025_A01
